Martín Amado - 19020
Juan Pablo Pineda - 19087

Referencia para análisis de datos: https://www.kaggle.com/competitions/digit-recognizer/data?select=test.csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import itertools
import matplotlib.cm as cm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Convolution2D, MaxPooling2D
from keras.layers import LSTM
from scipy import stats
from factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
from sklearn.decomposition import PCA
from keras.utils.np_utils import to_categorical
from sklearn.pipeline import make_pipeline
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from apyori import apriori

np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
sns.set(style='white', context='notebook', palette='deep')

# Ejercicio 1

In [ ]:
digitRec_train = pd.read_csv("./train.csv")
digitRec_test = pd.read_csv("./test.csv")

In [ ]:
digitRec_train

In [ ]:
digitRec_test

## Analisis Exploratorio

In [ ]:
baseMatrix = digitRec_train.to_numpy()

### Testing de imagenes

In [ ]:
from PIL import Image

img = Image.new('RGB', (28, 28), "black")
numbersMatrices = []

for x in baseMatrix[:5]:
    localmat = []
    for pixel in range(1, len(x), 28):
        localmat.append(list(x[pixel:pixel+28]))
    for row in range(0,28):
        for col in range(0,28):
            img.putpixel((col,row), (localmat[row][col], localmat[row][col], localmat[row][col]))
    numbersMatrices.append(localmat)
    img.show()


## Pre-procesamiento de los datos

In [ ]:
Y_train = digitRec_train['label']
X_train = digitRec_train.drop(['label'], axis=1)

g = sns.countplot(Y_train)

Y_train.value_counts()

In [ ]:
X_train.isnull().any().describe()

In [ ]:
digitRec_test.isnull().any().describe()

In [ ]:
X_train = X_train / 255.0
digitRec_test = digitRec_test / 255.0

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
digitRec_test = digitRec_test.values.reshape(-1,28,28,1)

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

### Primer Modelo de Red Neuronal

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Same',
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same',
                 activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation="softmax"))


### Optimizacion del modelo

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
epochs = 1 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                              epochs=epochs, validation_data=(X_val, Y_val),
                              verbose=2, steps_per_epoch=X_train.shape[0] // batch_size, callbacks=[learning_rate_reduction])


In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Predict the values from the validation dataset
Y_pred = model.predict(X_val)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = np.argmax(Y_val,axis = 1) 
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes) 
# plot the confusion matrix
plot_confusion_matrix(confusion_mtx, classes = range(10)) 

In [ ]:
errors = (Y_pred_classes - Y_true != 0)

Y_pred_classes_errors = Y_pred_classes[errors]
Y_pred_errors = Y_pred[errors]
Y_true_errors = Y_true[errors]
X_val_errors = X_val[errors]

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((28,28)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Probabilities of the wrong predicted numbers
Y_pred_errors_prob = np.max(Y_pred_errors,axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(Y_pred_errors, Y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = Y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

# Top 6 errors 
most_important_errors = sorted_dela_errors[-6:]

# Show the top 6 errors
display_errors(most_important_errors, X_val_errors, Y_pred_classes_errors, Y_true_errors)

### Predicciones

In [ ]:
results = model.predict(digitRec_test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
digitRec_test

### Modelo 2

#### Caracteristicas
- Filtros: 128
- Detector de propiedades de 3x3
- Movimiento de pixel en pixel
- Tamano imagenes 28x28, Lightness
- Matriz baja a 2x2

In [ ]:
modelo = Sequential()
modelo.add(Convolution2D(128,(3,3),strides=(1,1), input_shape=(28,28,1), activation='relu'))

In [ ]:
modelo.add(MaxPooling2D(pool_size=(2,2)))

#### Segunda pasada

In [ ]:
modelo.add(Convolution2D(64, (3, 3), strides=(1, 1), activation='relu'))
modelo.add(MaxPooling2D(pool_size=(2,2)))

##### Flattening y Dense

In [ ]:
modelo.add(Flatten())
modelo.add(Dense(256, activation='relu'))
modelo.add(Dense(1, activation='sigmoid'))

In [ ]:
modelo.summary()

In [ ]:
modelo.compile(optimizer='adam', loss='categorical_hinge', metrics='accuracy')

##### Entrenamiento

In [ ]:
epochs = 25
batch = 32
## Se vuelve a importar para evitar cambios realizados
digitRec_train = pd.read_csv("./train.csv")
digitRec_test = pd.read_csv("./test.csv")

Y_train = digitRec_train['label']
X_train = digitRec_train.drop(['label'], axis=1)

In [ ]:
X_train = X_train / 255.0
digitRec_test = digitRec_test / 255.0

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)
digitRec_test = digitRec_test.values.reshape(-1,28,28,1)

In [ ]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [ ]:
random_seed = 5
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

In [ ]:
H = modelo.fit(
  X_train,
  epochs=epochs
)

# Ejercicio 2

## Parte 1: Series de tiempo

### Consumos Diesel

Recaudacion y limpieza de datos

In [ ]:
consumption = pd.read_excel('./CONSUMO-2022-05.xlsx', skiprows=6)
consumption = consumption[['Fecha', 'Diesel']]
#omitimos los datos despues del 269 ya que no aportan relevancia
consumption = consumption[:269]

Desarrollo de la seria de tiempo

In [ ]:
trainSize = int(len(consumption) * 0.7)
trainConsumos = consumption[0:trainSize]
testConsumos = consumption[trainSize:len(consumption)]
trainConsumos = trainConsumos.set_index(['Fecha'])
testConsumos = testConsumos.set_index(['Fecha'])

In [ ]:
tsDiesel = trainConsumos['Diesel']
mediaMovil = tsDiesel.rolling(window=12).mean()
deMovil = tsDiesel.rolling(window=12).std()

In [ ]:
original = plt.plot(tsDiesel, color="blue", label="Original")
media = plt.plot(mediaMovil, color='red', label = 'Media Movil')
ds = plt.plot(deMovil,color='black', label = 'Desviación Estándar Móvil')
plt.legend(loc = 'best')
plt.title('Media y desviación estándar móvil')
plt.show(block=False)

In [ ]:
descomposicion = seasonal_decompose(tsDiesel)
descomposicion.plot()

In [ ]:
tsDiesel = tsDiesel.astype({'Diesel':'float'})
tsDieselLog = np.log(tsDiesel)
plt.plot(tsDieselLog)

In [ ]:
dfTest = adfuller(tsDiesel, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Valor Crítico (%s)'%key] = value
print(salidaDf)

In [ ]:
tsDieselDiff = tsDiesel.diff()
tsDieselDiff.fillna(0,inplace=True)
dfTest = adfuller(tsDieselDiff)
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
salidaDf

In [ ]:
plt.plot(tsDieselDiff)
plt.title('Diferenciacion de la serie')

In [ ]:
modelo111 = SARIMAX(tsDieselLog, order=(1,1,1), seasonal_order=(3,1,0,12), enforce_stationarity=False, enforce_invertibility=False)
resultado_m111 = modelo111.fit()
resultado_m111.summary().tables[1]

In [ ]:
resultado_m111.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
testConsumos.index[0]
pred = resultado_m111.get_prediction(start=testConsumos.index[0], dynamic=False)
pred_ci = pred.conf_int()
consumoIndexed = consumption.set_index('Fecha')
consumoIndexed = consumoIndexed['Diesel']
ax = consumoIndexed['2000':].plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between( pred_ci.iloc[:,0],
                pred_ci.iloc[:,1], color='k', alpha=.2)
#ax.set_xlabel('Date')
#ax.set_ylabel('Retail_sold')
plt.legend()
plt.show()

pred

## Parte 2: LSTM

In [ ]:
consumo = pd.read_excel('./CONSUMO-2022-05.xlsx', skiprows=6)
consumo = consumo[['Fecha', 'Diesel']]
#omitimos los datos despues del 269 ya que no aportan relevancia
consumo = consumo[:269]
consumo = consumo.set_index('Fecha')
consumo.head()

In [ ]:
plt.plot(consumo)
plt.gca().set(title="Consumo de Diesel por mes por año", xlabel="Fecha", ylabel="Consumo")
plt.show()

Estacionarizar para uso de LSTM

In [ ]:
# Se calcula la media móvil y la desviación estandar móvil de los últimos 12 meses.
mediaMovil = consumo.rolling(window=12).mean()
deMovil = consumo.rolling(window=12).std()
# Se grafican los resultados.
original = plt.plot(consumo, color="blue", label="Original")
media = plt.plot(mediaMovil, color='red', label = 'Media Movil')
ds = plt.plot(deMovil,color='black', label = 'Desviación Estándar Móvil')
plt.legend(loc = 'best')
plt.title('Media y desviación estándar móvil')
plt.show(block=False)

In [ ]:
descomposicion = seasonal_decompose(consumo)
descomposicion.plot()

Existe tendencia

In [ ]:
print('Resultados del Test de Dickey Fuller')
dfTest = adfuller(consumo, autolag='AIC')
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
print('Resultados del Test de Dickey Fuller para una diferenciación de la serie')
diesel_diff = consumo.diff()
diesel_diff.fillna(0,inplace=True)
dfTest = adfuller(diesel_diff)
salidaDf = pd.Series(dfTest[0:4], index=['Estadístico de prueba','p-value','# de retardos usados','# de observaciones usadas'])
for key,value in dfTest[4].items():
        salidaDf['Critical Value (%s)'%key] = value
print(salidaDf)

In [ ]:
plt.plot(diesel_diff)

Necesidad de diferenciacion

In [ ]:
scaler = StandardScaler()
diesel_scaled = scaler.fit_transform(diesel_diff) 
diesel_scaled[1:7]

60% entrenamiento, 20% validacion y prueba

In [ ]:
train_diesel = round(0.6*len(diesel_scaled))
val_test_diesel = round(0.2*len(diesel_scaled))
test = diesel_scaled[(train_diesel + val_test_diesel) - 1:]
validation = diesel_scaled[(train_diesel):train_diesel + val_test_diesel + 1]
train = diesel_scaled[0:train_diesel]
train = np.insert(train, 0, 0)
train = np.reshape(train, (train.shape[0], 1))

Transformacion a serie supervisada

In [ ]:
from operator import concat


def supervisada(serie,retrasos = 1):
    serie_x = []
    serie_y = []
    for i in range(len(serie)-retrasos):
        valor = serie[i:(i+retrasos),0]
        valor_sig = serie[i+retrasos,0]
        serie_x.append(valor)
        serie_y.append(valor_sig)
    return np.array(serie_x), np.array(serie_y)

x_train,y_train = supervisada(train)
x_val,y_val = supervisada(validation)
x_test,y_test = supervisada(test)

#### Creacion de modelo

##### Modelo 1

Matrices de 3 dimensiones

In [ ]:
x_train = np.reshape(x_train,(x_train.shape[0],1,1))
x_val = np.reshape(x_val, (x_val.shape[0],1,1))
x_test = np.reshape(x_test, (x_test.shape[0],1,1))

Modelo con una capa LSTM

In [ ]:
modelo1 = Sequential()
lote = 1
unidades =  1
paso = 1
caracteristicas = 1 #es univariada
modelo1.add(LSTM(lote, batch_input_shape=(lote,paso,caracteristicas),stateful=True))
modelo1.add(Dense(1))
modelo1.summary()

In [ ]:
modelo1.compile(loss='mean_squared_error',optimizer="rmsprop")

Entrenamiento

In [ ]:
epocas = 50
history= modelo1.fit(
    x = x_train,
    y = y_train,
    batch_size = lote,
    epochs = epocas,
    shuffle = False,
    validation_data = (x_val,y_val),
    verbose=1
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='center')

In [ ]:
print("Pérdida en Entrenamiento")
modelo1.evaluate(
    x = x_train,
    y = y_train
)
print("Pérdida en Validación")
modelo1.evaluate(
    x = x_val,
    y = y_val
)
print("Pérdida en Prueba")
modelo1.evaluate(
    x = x_test,
    y = y_test
)

Prediccion modelo 1

In [ ]:
prediccion_val = []

def prediccion_fun(data,modelo, batch_size,scaler,dif=False,dif_cant=1, Series = None , n=1):
    prediccion = [0]* (len(data))
    i=0
    for X in data:
        X = np.reshape(X,(1,1,1))
        yhat = modelo1.predict(X, batch_size=batch_size,verbose=0)
        # invert scaling
        yhat = scaler.inverse_transform(yhat)
        if dif:
             # invert differencing
            yhat  = yhat + Series[(n+dif_cant*i)]
        # store
        prediccion[i]=yhat[0][0]
        i = i+1
    return prediccion
prediccion_val = prediccion_fun(x_val,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel)
prediccion_test = prediccion_fun(x_test,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel+val_test_diesel)


In [ ]:
df_val = pd.DataFrame(prediccion_val,index=consumo[(train_diesel):train_diesel+val_test_diesel].index)
df_test = pd.DataFrame(prediccion_test,index=consumo[train_diesel+len(df_val):len(consumo)].index)


plt.plot(consumo)
plt.plot(df_val)
plt.plot(df_test)

##### Modelo 2

Para el modelo 2, las unidades cambiaran

In [ ]:
modelo2 = Sequential()
lote = 1
unidades =  5
paso = 1
caracteristicas = 1 #es univariada
modelo2.add(LSTM(lote, batch_input_shape=(lote,paso,caracteristicas),stateful=True))
modelo2.add(Dense(1))
modelo2.summary()

In [ ]:
modelo2.compile(loss='mean_squared_error',optimizer="rmsprop")

In [ ]:
epocas = 50
history= modelo2.fit(
    x = x_train,
    y = y_train,
    batch_size = lote,
    epochs = epocas,
    shuffle = False,
    validation_data = (x_val,y_val),
    verbose=1
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='center')

In [ ]:
print("Pérdida en Entrenamiento")
modelo2.evaluate(
    x = x_train,
    y = y_train
)
print("Pérdida en Validación")
modelo2.evaluate(
    x = x_val,
    y = y_val
)
print("Pérdida en Prueba")
modelo2.evaluate(
    x = x_test,
    y = y_test
)

In [ ]:
prediccion_val = []

def prediccion_fun(data,modelo, batch_size,scaler,dif=False,dif_cant=1, Series = None , n=1):
    prediccion = [0]* (len(data))
    i=0
    for X in data:
        X = np.reshape(X,(1,1,1))
        yhat = modelo1.predict(X, batch_size=batch_size,verbose=0)
        # invert scaling
        yhat = scaler.inverse_transform(yhat)
        if dif:
             # invert differencing
            yhat  = yhat + Series[(n+dif_cant*i)]
        # store
        prediccion[i]=yhat[0][0]
        i = i+1
    return prediccion
prediccion_val = prediccion_fun(x_val,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel)
prediccion_test = prediccion_fun(x_test,modelo1, 1,scaler,dif=True,dif_cant=1, Series = consumo.values , n=train_diesel+val_test_diesel)


In [ ]:
df_val = pd.DataFrame(prediccion_val,index=consumo[(train_diesel):train_diesel+val_test_diesel].index)
df_test = pd.DataFrame(prediccion_test,index=consumo[train_diesel+len(df_val):len(consumo)].index)


plt.plot(consumo)
plt.plot(df_val)
plt.plot(df_test)



#### Conclusiones 

Los modelos LSTM fueron de mayor utilidad, pues presentaron una prediccion, a diferencia del utilizado anteriormente.

El segundo modelo LSTM es mejor, debido a los valores de perdida que presenta en entrenamiento validacion y prueba